<a href="https://colab.research.google.com/github/SubhradeepSS/Data-Modeling/blob/Unet-model/Unet%20Model/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install segmentation-models
import segmentation_models as sm

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [0]:
!pip install -q kaggle

In [0]:
from google.colab import files

files.upload()

In [0]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [0]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d emmarex/plantdisease

 98% 643M/658M [00:03<00:00, 203MB/s]
100% 658M/658M [00:03<00:00, 211MB/s]


In [0]:
%%capture
!unzip plantdisease.zip

In [8]:
!ls PlantVillage

Pepper__bell___Bacterial_spot  Tomato_Late_blight
Pepper__bell___healthy	       Tomato_Leaf_Mold
Potato___Early_blight	       Tomato_Septoria_leaf_spot
Potato___healthy	       Tomato_Spider_mites_Two_spotted_spider_mite
Potato___Late_blight	       Tomato__Target_Spot
Tomato_Bacterial_spot	       Tomato__Tomato_mosaic_virus
Tomato_Early_blight	       Tomato__Tomato_YellowLeaf__Curl_Virus
Tomato_healthy


In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_gen.flow_from_directory("PlantVillage", batch_size = 64, class_mode = "categorical", shuffle = True)
class_dictionary = train_generator.class_indices
class_dictionary

Found 20638 images belonging to 15 classes.


{'Pepper__bell___Bacterial_spot': 0,
 'Pepper__bell___healthy': 1,
 'Potato___Early_blight': 2,
 'Potato___Late_blight': 3,
 'Potato___healthy': 4,
 'Tomato_Bacterial_spot': 5,
 'Tomato_Early_blight': 6,
 'Tomato_Late_blight': 7,
 'Tomato_Leaf_Mold': 8,
 'Tomato_Septoria_leaf_spot': 9,
 'Tomato_Spider_mites_Two_spotted_spider_mite': 10,
 'Tomato__Target_Spot': 11,
 'Tomato__Tomato_YellowLeaf__Curl_Virus': 12,
 'Tomato__Tomato_mosaic_virus': 13,
 'Tomato_healthy': 14}

In [10]:
train_generator.labels

array([ 0,  0,  0, ..., 14, 14, 14], dtype=int32)

In [11]:
x, y = train_generator.next()
print(x.shape)
print(y.shape)

(64, 256, 256, 3)
(64, 15)


In [12]:
model_initial = sm.Unet('resnet50', input_shape=(256,256,3), encoder_weights='imagenet',classes=15)
model_initial.summary()

94593024/94592056 [==============================] - 2s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 256, 256, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 128, 128, 64) 9408        zero_padding2d_1[0][0]           
___________________________

In [14]:
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
average_pool = GlobalAveragePooling2D()(model_initial.output)
fc = Dense(256 , activation = "relu")(average_pool)
d = Dropout(0.4)(fc)
fc_ = Dense(15 , activation= "softmax")(d)

model_final = Model(input = model_initial.input, output = fc_)
model_final.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 256, 256, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 128, 128, 64) 9408        zero_padding2d_1[0][0]           
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("da..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [0]:
model_final.compile(
    loss = "categorical_crossentropy", optimizer ="Adam", metrics = ["accuracy"] 
)

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)

(51, 256, 256, 3) (51, 15) (13, 256, 256, 3) (13, 15)


In [20]:
model_final.fit(
   x=x_train,
   y=y_train,
   batch_size=16,
   epochs=100,
   validation_data=(x_val, y_val),
)

Train on 51 samples, validate on 13 samples
Epoch 1/100
51/51 [==============================] - 1s 26ms/step - loss: 1.0062 - accuracy: 0.7255 - val_loss: 9.7669 - val_accuracy: 0.0000e+00
Epoch 2/100
51/51 [==============================] - 1s 25ms/step - loss: 1.0337 - accuracy: 0.6667 - val_loss: 9.7382 - val_accuracy: 0.0000e+00
Epoch 3/100
51/51 [==============================] - 1s 25ms/step - loss: 0.8921 - accuracy: 0.6863 - val_loss: 9.2963 - val_accuracy: 0.0000e+00
Epoch 4/100
51/51 [==============================] - 1s 26ms/step - loss: 0.9206 - accuracy: 0.7451 - val_loss: 5.6674 - val_accuracy: 0.0769
Epoch 5/100
51/51 [==============================] - 1s 25ms/step - loss: 0.9925 - accuracy: 0.6863 - val_loss: 7.7291 - val_accuracy: 0.0000e+00
Epoch 6/100
51/51 [==============================] - 1s 26ms/step - loss: 0.8662 - accuracy: 0.7255 - val_loss: 9.6958 - val_accuracy: 0.0000e+00
Epoch 7/100
51/51 [==============================] - 1s 26ms/step - loss: 0.9468 - a

In [0]:
model_final.save('Unet_model.h5')